# Plan your trip with Kayak

In this notebook we illustrate data loading from the database to display on maps. We show two maps, one representing the weather at the different places, the other giving hotels information at such places.

Contents
--------
1. [Connection to the database](#database)
2. [Weather map](#weather)
2. [Hotels map](#hotels)

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from sqlalchemy import create_engine, URL
from sqlalchemy import text


NEONDB_ACCESS_KEYS = './neondb_access_keys.key'

## <a name='database'></a>Connection to the database

We fetch the relevant data from the database using SQL queries.

In [2]:
## get credentials
with open(NEONDB_ACCESS_KEYS, 'rt', encoding='utf-8') as f:
    pghost = f.readline().split("'")[1]
    pgdatabase = f.readline().split("'")[1]
    pguser = f.readline().split("'")[1]
    pgpassword = f.readline().split("'")[1]

url = URL.create(
    'postgresql+psycopg',
    username=pguser,
    password=pgpassword,
    host=pghost,
    database=pgdatabase,
)

## setup SQL engine
engine = create_engine(url, echo=False)

## <a name='weather'></a>Weather map

The following code plots the weather map shown below.

<img src="media/weather_map.png" alt="Weather map" width="600"/>

In [3]:
## Query data from the database
weather_df_query = """
SELECT
  loc.name AS name,
  loc.latitude AS latitude,
  loc.longitude AS longitude,
  wi.min_temperature_cels AS min_temp,
  wi.max_temperature_cels AS max_temp,
  wi.sunshine_duration_h AS sunshine,
  wi.precipitation_sum_mm AS precipitations
FROM
  locations AS loc
  JOIN weather_indicators AS wi ON loc.location_id = wi.location_id
"""

weather_df = pd.read_sql(text(weather_df_query), con=engine)

In [4]:
from sqlalchemy.orm import Session

with Session(engine) as s:
    s.rollback()

In [5]:
## Plot the data on map
fig_weather = go.Figure()

fig_weather.update_layout(title='Weather map',
                          height=800,
                          width=900,
                          map_style='open-street-map')

fig_weather.add_trace(
    go.Scattermap(
        lat=weather_df['latitude'],
        lon=weather_df['longitude'],
        marker=go.scattermap.Marker(
            color=weather_df['sunshine'],
            colorbar={'title': 'sunshine<br>duration', 'thickness': 30,
                      'ticks': 'inside', 'ticklen': 8, 'tickwidth': 2},
            size=2.2*weather_df['max_temp'],
        ),
        text=weather_df['name'],
        customdata=weather_df.iloc[:, -4:],
        hovertemplate=(
            '<b>%{text}</b><br><br>'
            'Min temperature (°C): %{customdata[0]:.1f}<br>'
            'Max temperature (°C): %{customdata[1]:.1f}<br>'
            'Sunshine duration (h): %{customdata[2]:,.1f}<br>'
            'Precipitations (mm): %{customdata[3]:.2f}'
            '<extra></extra>'
        ),
    )
)

fig_weather.show()

## <a name='hotels'></a>Hotels 

The following code plots the hotels map shown below. The geographic coordinates of the hotels were not available anymore on booking.com. The map therefore does not indicate each individual hotel, but rather an hover text with the 5 best rated hotels at the given place.

<img src="media/hotels_map.png" alt="Hotels map" width="600"/>

In [6]:
locations_df = pd.read_sql(text('SELECT location_id, name, latitude, longitude FROM locations'), con=engine)
hotels_df = pd.read_sql(text('SELECT location_id, name, rating, georating FROM hotels'), con=engine)

In [7]:
hotels_loc = []
for location_id in locations_df['location_id']:
    loc = (hotels_df['location_id'] == location_id)
    hotels_loc.append(hotels_df.loc[loc].sort_values(['rating', 'georating'], ascending=False).head(5))
hotels_loc = [np.ravel(h.loc[:, ['name', 'rating', 'georating']].to_numpy()) for h in hotels_loc]

In [8]:
fig_hotels = go.Figure()

fig_hotels.update_layout(title='Hotels map',
                          height=800,
                          width=900,
                          map_style='open-street-map')

fig_hotels.add_trace(
    go.Scattermap(
        lat=locations_df['latitude'],
        lon=locations_df['longitude'],
        marker=go.scattermap.Marker(size=15,),
        text=locations_df['name'],
        customdata=hotels_loc,
        hovertemplate=(
            '<b>%{text}</b><br><br>'
            'Hotel name, rating, geo rating<br>'
            '%{customdata[0]}: %{customdata[1]:.1f}, %{customdata[2]:.1f}<br>'
            '%{customdata[3]}: %{customdata[4]:.1f}, %{customdata[5]:.1f}<br>'
            '%{customdata[6]}: %{customdata[7]:.1f}, %{customdata[8]:.1f}<br>'
            '%{customdata[9]}: %{customdata[10]:.1f}, %{customdata[11]:.1f}<br>'
            '%{customdata[12]}: %{customdata[13]:.1f}, %{customdata[14]:.1f}<br>'
            '<extra></extra>'
        ),
    )
)

fig_hotels.show()

In [9]:
## close database connection gracefully
engine.dispose()